In [ ]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           HypervolumeMetric, EpsilonIndicatorMetric, Scenario)
from ema_workbench.em_framework.optimization import EpsilonProgress
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from itertools import product

from problem_formulation import get_model_for_problem_formulation

In [ ]:
model, steps = get_model_for_problem_formulation(2)
with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.optimize(nfe=5000, epsilons=[100,100,100,100,100])

  0%|                                                 | 0/5000 [00:00<?, ?it/s]

In [8]:
def load_scenarios(filepath):
    df = pd.read_csv(filepath)
    scenarios = []
    for _, row in df.iterrows():
        name = row['scenario']
        parameters = row.drop(labels='scenario').to_dict()
        scenarios.append(Scenario(name, **parameters))

    return scenarios 
scenarios = load_scenarios('./data/Selected_Scenarios.csv')

In [9]:
def find_optimal_epsilons(model, scenarios=None, nfe=10000, convergence_threshold=0.05,
                          n_replicates=3, epsilons_range=None, convergence_interval=1000):
    """
    Perform epsilon sensitivity analysis with scenario support.
    
    Parameters:
    -----------
    model : EMA Model
        The model to optimize
    scenarios : list or None
        List of scenarios to evaluate against (if None, uses default optimization)
    nfe : int
        Number of function evaluations per optimization run
    convergence_threshold : float
        Threshold for epsilon progress convergence
    n_replicates : int
        Number of replicates for each epsilon configuration
    epsilons_range : dict or None
        Dictionary with outcome names as keys and epsilon ranges as values
    convergence_interval : int
        Interval for checking convergence
    
    Returns:
    --------
    dict
        Dictionary containing results organized by scenario
    """
    ema_logging.log_to_stderr(ema_logging.INFO)

    # Handle default scenario case
    if scenarios is None:
        scenarios = [None]

    # Initialize results structure
    results = {
        'scenarios': [],
        'epsilons_tested': [],
        'convergence_data': [],
        'hypervolume': [],
        'pareto_sets': []
    }

    # Determine epsilon ranges if not provided
    if epsilons_range is None:
        with MultiprocessingEvaluator(model) as evaluator:
            results = evaluator.perform_experiments(1000)
        _, outcomes = results
        epsilons_range = {}
        for outcome_name, values in outcomes.items():
            outcome_range = np.percentile(
                values, 95) - np.percentile(values, 5)
            epsilons_range[outcome_name] = np.linspace(0.01 * outcome_range,
                                                       0.1 * outcome_range, 5)

    # Generate all epsilon combinations to test
    epsilon_combinations = list(product(*epsilons_range.values()))

    for scenario in scenarios:
        scenario_results = {
            'epsilons_tested': [],
            'convergence_data': [],
            'hypervolume': [],
            'pareto_sets': []
        }

        for epsilons in epsilon_combinations:
            current_epsilons = dict(zip(epsilons_range.keys(), epsilons))

            rep_convergence = []
            rep_hypervolume = []
            rep_pareto = []

            for rep in range(n_replicates):
                with MultiprocessingEvaluator(model) as evaluator:
                    convergence = [EpsilonProgress()]
                    hypervolume = [HypervolumeMetric(model.outcomes, scenario)]

                    result = evaluator.optimize(
                        nfe=nfe,
                        epsilons=current_epsilons,
                        convergence=convergence + hypervolume,
                        convergence_threshold=convergence_threshold,
                        convergence_interval=convergence_interval,
                        reference=scenario
                    )

                    rep_convergence.append(convergence[0].epsilon_progress)
                    rep_hypervolume.append(hypervolume[0].hypervolume[-1])
                    rep_pareto.append(result)

            scenario_results['epsilons_tested'].append(current_epsilons)
            scenario_results['convergence_data'].append(
                np.mean(rep_convergence, axis=0))
            scenario_results['hypervolume'].append(np.mean(rep_hypervolume))
            scenario_results['pareto_sets'].append(rep_pareto)

        # Store scenario results
        results['scenarios'].append(scenario)
        results['epsilons_tested'].append(scenario_results['epsilons_tested'])
        results['convergence_data'].append(
            scenario_results['convergence_data'])
        results['hypervolume'].append(scenario_results['hypervolume'])
        results['pareto_sets'].append(scenario_results['pareto_sets'])

    return results

In [ ]:
# Load  model
dike_model, planning_steps = get_model_for_problem_formulation(2)

# Run epsilon sensitivity analysis
results = find_optimal_epsilons(dike_model)

[MainProcess/INFO] pool started with 24 workers
[MainProcess/INFO] performing 1000 scenarios * 1 policies * 1 model(s) = 1000 experiments
  0%|                                                 | 0/1000 [00:00<?, ?it/s]